In [56]:
# Pkg.add("Iterators")

In [57]:
# Pkg.update()

In [58]:
using JuMP, Cbc
using Combinatorics, Iterators
using StatsBase

In [135]:
I_skus = [1,2,3]
m = length(I_skus)

3

In [136]:
K_warehouses = [1,2]
k = length(K_warehouses)

2

In [137]:
max_inventory = 5
inventory = rand(1:max_inventory, m, k)

3×2 Array{Int64,2}:
 3  3
 2  2
 2  1

In [144]:
min_order_size = 2
max_order_size = m

3

In [145]:
sku_combos = [collect(combinations(I_skus, o)) for o in min_order_size:max_order_size];

In [146]:
J_orders = []
for i in 1:length(sku_combos)
    append!(J_orders, sku_combos[i])
end

In [147]:
J_orders

4-element Array{Any,1}:
 [1, 2]   
 [1, 3]   
 [2, 3]   
 [1, 2, 3]

In [157]:
j =length(J_orders)

4

In [50]:
# wh_allocation = [.3, .7]
# orders_wh = [sample(orders, Int(round(length(orders) * wh, 0)), replace=false) for wh in wh_allocation]
# # orders_wh = [[o, o in wc ? 1 : 2] for o in orders]

In [173]:
I_j = [unique(s) for s in J_orders]

4-element Array{Array{Int64,1},1}:
 [1, 2]   
 [1, 3]   
 [2, 3]   
 [1, 2, 3]

In [174]:
N = [collect(subsets(j)) for j in J_orders]

4-element Array{Array{Array{Int64,1},1},1}:
 Array{Int64,1}[Int64[], [1], [2], [1, 2]]                                
 Array{Int64,1}[Int64[], [1], [3], [1, 3]]                                
 Array{Int64,1}[Int64[], [2], [3], [2, 3]]                                
 Array{Int64,1}[Int64[], [1], [2], [1, 2], [3], [1, 3], [2, 3], [1, 2, 3]]

In [175]:
n = sum([length(n_j) for n_j in N])

20

In [176]:
N[3][3]

1-element Array{Int64,1}:
 3

In [154]:
for n_j in N
    for l in n_j
        if length(l) > 0
            @show l
        end
    end
end

l = [1]
l = [2]
l = [1, 2]
l = [1]
l = [3]
l = [1, 3]
l = [2]
l = [3]
l = [2, 3]
l = [1]
l = [2]
l = [1, 2]
l = [3]
l = [1, 3]
l = [2, 3]
l = [1, 2, 3]


In [195]:
m, n, j

(3, 20, 4)

In [191]:
N[1][2] == [1]

true

In [181]:
n

20

In [194]:
A = [N[o][l] for o=1:j, l=1:4, i=1:m]

4×4×3 Array{Array{Int64,1},3}:
[:, :, 1] =
 Int64[]  [1]  [2]  [1, 2]
 Int64[]  [1]  [3]  [1, 3]
 Int64[]  [2]  [3]  [2, 3]
 Int64[]  [1]  [2]  [1, 2]

[:, :, 2] =
 Int64[]  [1]  [2]  [1, 2]
 Int64[]  [1]  [3]  [1, 3]
 Int64[]  [2]  [3]  [2, 3]
 Int64[]  [1]  [2]  [1, 2]

[:, :, 3] =
 Int64[]  [1]  [2]  [1, 2]
 Int64[]  [1]  [3]  [1, 3]
 Int64[]  [2]  [3]  [2, 3]
 Int64[]  [1]  [2]  [1, 2]

In [200]:
A = [Int(i in (N[o][l])) for l=1:4, i=1:m, o=1:j]

4×3×4 Array{Int64,3}:
[:, :, 1] =
 0  0  0
 1  0  0
 0  1  0
 1  1  0

[:, :, 2] =
 0  0  0
 1  0  0
 0  0  1
 1  0  1

[:, :, 3] =
 0  0  0
 0  1  0
 0  0  1
 0  1  1

[:, :, 4] =
 0  0  0
 1  0  0
 0  1  0
 1  1  0

In [189]:
model = Model(solver=CbcSolver())

# decision variable (binary): whether to ship subset C_l from warehouse k
@variable(model, y[1:n, 1:j, 1:k], Bin)

# # Objective: minimize number of shipments
@objective(model, Min, sum(y))

# # Constraint: has to cover all warehouses
# # (.>= is the element-wise dot comparison operator)
# @constraint(model, A*y .>= 1)

model

Minimization problem with:
 * 0 linear constraints
 * 160 variables: 160 binary
Solver is CbcMathProg

We have an additional constraint that at least 1 warehouse should be within 10 miles of distribution center 1, but our activity matrix $A$ already covers that, so technically we do not need this explicit constraint.

In [125]:
@constraint(model, y[1] + y[2] >= 1)

y[1] + y[2] ≥ 1

In [126]:
# Solve problem using MIP solver
status = solve(model)

:Optimal

In [137]:
println("Total # of warehouses: ", getobjectivevalue(model))

println("Build warehouses at distribution center(s):")

[i for i=1:m if getvalue(y[i]) == 1 ]

Total # of warehouses: 2.0
Build warehouses at distribution center(s):


2-element Array{Int64,1}:
 2
 3